In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [2]:
df = pd.read_csv("metadata_updated.csv")

In [3]:
updated_df = df.copy()

In [4]:
updated_df['categories'] = ''
updated_df['publish_time'] = ''

In [ ]:
# Add category (and subcategory) and timestamp to df

for i in range(len(updated_df)):
    if i % 100 == 0:
        print(str(i / len(updated_df)) + "%", "done")
    
    link = updated_df.iat[i, 4]
    
    if link.startswith("https://anchor.fm"):
        
        try:
            r = requests.get(link, timeout = 5)
        
            xml_data = r.content


            soup = BeautifulSoup(xml_data, "html5lib")

            if soup.body.string != "Not Found":
                child = soup.find("channel")

                try:
                    category = child.find('itunes:category')["text"]
                except:
                    category = ""

                try:
                    subchild = soup.find('itunes:category').findChildren()[0]
                    subcategory = subchild["text"]
                except:
                    subcategory = ""

                try:
                    time = child.find('itunes:category').find('pubdate').string
                    time = datetime.strptime(time, '%a, %d %b %Y %H:%M:%S GMT')
                    # time = (time.month, time.day, time.year)
                except:
                    time = ""

                if category == "":
                    categories = ""
                elif subcategory != "":
                    categories = [category, subcategory]
                else:
                    categories = [category]
                    
                updated_df.iat[i, 6] = categories
                updated_df.iat[i, 7] = time
                    
        except:
            
            print("Invalid Link:", link)
            

In [17]:
updated_df

,show_name,show_uri,show_description,publisher,rss_link,avg_duration,categories,publish_time
0,The Grier Podcast,spotify:show:1spDjPLku0UCnir7H2hEG4,A podcast recorded and created by the students...,Bryan Beckel,https://anchor.fm/s/11399a34/podcast/rss,21.389133,[Education],2022-02-21 12:00:00
1,"""ACIM"" A Course In Miracles For The Millennium",spotify:show:5wb7vWvrCtgsY3jMvIAOXW,"""A study group for the transformation of life ...",Rafael Hidalgo,https://anchor.fm/s/25197c4/podcast/rss,87.984350,[Religion & Spirituality],2019-02-21 19:27:18
2,AfroLatina_BeautifitlyBalanced,spotify:show:3zbdWAjWe2sp91t3Ly1j49,Health+SELF Love+Spotless Clean Home! Thats ev...,The AfroLatina_BeautifitlyBalanced,https://anchor.fm/s/9571594/podcast/rss,19.906850,"[Health & Fitness, Mental Health]",2022-07-08 03:24:43
3,Apostles of Tomorrow,spotify:show:31r4WwUvNQ0CHGU2xobnZP,"A podcast to help fellow Gen Z Apostolics,To f...",Your host Garrett,https://anchor.fm/s/ddebcac/podcast/rss,29.358917,"[Religion & Spirituality, Spirituality]",2020-10-29 17:00:00
4,Best Blogging Courses,spotify:show:18jt1Biy9vlS449Gz3j9oE,Experts have compiled this list of Best Blogg...,joy ben,https://anchor.fm/s/122df548/podcast/rss,4.588850,,
...,...,...,...,...,...,...,...,...
18251,free your mind: podcast,spotify:show:2DqGCojWyryoDFq0argY9E,Sharing the journey of life & how to set your ...,Ima,https://anchor.fm/s/e94e5e0/podcast/rss,10.529867,"[Education, Self-Improvement]",2021-03-02 03:49:00
18252,The Best Comes from the worst,spotify:show:31aNxmwRKE2zlQbYVglQxe,In order to live your best life you must exper...,James Taylor,https://anchor.fm/s/b2b7450/podcast/rss,16.416900,[Religion & Spirituality],2019-10-14 17:02:45
18253,RED CUP VIBEz,spotify:show:5omDM65kJwYhlJdAiA86fn,Sipping too many adult beverages out of a red ...,BURBERRYLYTE,https://anchor.fm/s/12949c94/podcast/rss,36.678233,"[Comedy, Improv]",2020-01-19 10:15:58
18254,PositiveVibes 24/7,spotify:show:5ueyekcDdst5X5ntxReUz8,PositiveVibes Only,PositiveVibes,https://anchor.fm/s/26f0a70/podcast/rss,3.333292,[Education],2019-02-04 20:59:58


In [18]:
remove_null = updated_df.copy()

In [20]:
remove_null = remove_null[remove_null['categories'] != ""]
remove_null = remove_null[remove_null['publish_time'] != ""]

In [22]:
remove_null.to_csv("genres.csv")